# Train a DNN on MNIST

In [1]:
import keras as K
from keras.models import Sequential
from keras.layers import Dense, Reshape, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.callbacks import TensorBoard
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

Using TensorFlow backend.


## Load the data

In [2]:
# TODO use the data from the Kaggle competition
# (but it comes as csv and ugly)

In [3]:
mnist = input_data.read_data_sets('/tmp/mnist', one_hot=True)
train_img = mnist.train.images
train_lab = mnist.train.labels
test_img = mnist.test.images
test_lab = mnist.test.labels

img_size = (28,28,1)
input_size = 784

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/mnist/t10k-labels-idx1-ubyte.gz


# Construct the model

In [4]:
model = Sequential()
model.add(Reshape(img_size, input_shape=(input_size,)))
model.add(Conv2D(32,(5,5), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(5,5), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
__________

### Add tensorboard callback

In [5]:
tensorboard_callback = TensorBoard(log_dir='/tmp/mnist/logs',
                                   write_images=True)

# Train the model

In [6]:
batch_size = 128
epochs = 20

model.fit(x=train_img,
          y=train_lab,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(test_img, test_lab),
          callbacks=[tensorboard_callback])

Train on 55000 samples, validate on 10000 samples
Epoch 1/20
55000/55000 [==============================] - 5s 98us/step - loss: 0.1595 - acc: 0.9513 - val_loss: 0.0371 - val_acc: 0.9880
Epoch 2/20
55000/55000 [==============================] - 5s 89us/step - loss: 0.0442 - acc: 0.9860 - val_loss: 0.0348 - val_acc: 0.9892
Epoch 3/20
55000/55000 [==============================] - 5s 89us/step - loss: 0.0312 - acc: 0.9899 - val_loss: 0.0304 - val_acc: 0.9910
Epoch 4/20
55000/55000 [==============================] - 5s 90us/step - loss: 0.0235 - acc: 0.9930 - val_loss: 0.0301 - val_acc: 0.9912
Epoch 5/20
55000/55000 [==============================] - 5s 90us/step - loss: 0.0191 - acc: 0.9945 - val_loss: 0.0264 - val_acc: 0.9914
Epoch 6/20
55000/55000 [==============================] - 5s 90us/step - loss: 0.0156 - acc: 0.9950 - val_loss: 0.0366 - val_acc: 0.9902
Epoch 7/20
55000/55000 [==============================] - 5s 91us/step - loss: 0.0130 - acc: 0.9959 - val_loss: 0.0391 - val_acc